# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

print("total rows:", len(full_data_rows_list))

# Creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


total rows: 8056


In [4]:
# Number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# Make connection to Cassandra instance
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# Create session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create Keyspace udacity
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to udacity
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1

For this first query since we are interested in a specific session and in which place the particular song was listened to it makes sense to include both session_id and item_in_session as primary key. We will call this table the song_library because it comprises songs metadata.

In [8]:
# Query
query_songs = "SELECT artist_name, song_title, song_length FROM songs_library WHERE session_id = 338 AND item_in_session = 4"

In [9]:
# Input file
file = 'event_datafile_new.csv'

# Open file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    # Create table
    query = "CREATE TABLE IF NOT EXISTS songs_library "
    query = query + """(
        artist_name text, 
        song_title text, 
        song_length decimal, 
        session_id int, 
        item_in_session int, 
        PRIMARY KEY (session_id, item_in_session)
    )"""
    
    try:
        session.execute(query)
    except Exception as e:
        print(e)
        
    # Upload data Line by Line
    for line in csvreader:
        
        query = """INSERT INTO songs_library (
            artist_name, 
            song_title, 
            song_length, 
            session_id, 
            item_in_session
        )"""
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        try:
            # Some attributes need to be casted to proper types
            session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))    
        except Exception as e:
            print(e)

Once the table has been created and the data inserted we can query the database

In [10]:
try:
    rows = session.execute(query_songs)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist_name, row.song_title, row.song_length, sep="--")

Faithless--Music Matters (Mark Knight Dub)--495.3073


### Query 2

For the second query what we want to know which songs did a particular user listented to during a session which means we need the user_id and session_id as members of our primary key. But we also want it sorted by item_in_session so we must include this clustering column too. We will call this table user_playlist because it summarizes a particular user listening habits.

In [11]:
query_playlist = "SELECT artist_name, song_title, first_name, last_name FROM user_playlist WHERE user_id = 10 AND session_id = 182"

# Open file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    # Create table
    query = "CREATE TABLE IF NOT EXISTS user_playlist "
    query = query + """(
        artist_name text, 
        song_title text, 
        first_name text, 
        last_name text, 
        user_id int,
        session_id int, 
        item_in_session int, 
        PRIMARY KEY (user_id, session_id, item_in_session)
    )"""
    try:
        session.execute(query)
    except Exception as e:
        print(e)
    
    # Upload data Line by Line
    for line in csvreader:
        
        query = """INSERT INTO user_playlist (
            artist_name, 
            song_title, 
            first_name, 
            last_name, 
            user_id, 
            session_id, 
            item_in_session
        )"""
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        try:
            # Some attributes need to be casted to proper types
            session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))    
        except Exception as e:
            print(e)
        
# Get result from query to test table
try:
    rows = session.execute(query_playlist)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist_name, row.song_title, row.first_name, row.last_name, sep="--")

Down To The Bone--Keep On Keepin' On--Sylvie--Cruz
Three Drives--Greece 2000--Sylvie--Cruz
Sebastien Tellier--Kilometer--Sylvie--Cruz
Lonnie Gordon--Catch You Baby (Steve Pitron & Max Sanna Radio Edit)--Sylvie--Cruz


### Query 3

For the final query we are interested in users that have listened to particular songs. We will call this table song_fans because we want the distinct users for a song. In this case it is obvious we must include song as part of our primary key, however it we make it our solely component users names will be overwritten with each insert, for this reason we have to include the user_id in our primary key. 

In [12]:
query_fans = "SELECT first_name, last_name FROM song_fans WHERE song_title = 'All Hands Against His Own'"

# Open file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    # Create table
    query = "CREATE TABLE IF NOT EXISTS song_fans "
    query = query + """(
        first_name text, 
        last_name text, 
        song_title text,
        user_id int, 
        PRIMARY KEY (song_title, user_id)
    )"""
    try:
        session.execute(query)
    except Exception as e:
        print(e)

    # Upload data Line by Line
    for line in csvreader:
        
        query = """INSERT INTO song_fans ( 
            first_name, 
            last_name, 
            song_title,
            user_id
        )"""
        query = query + " VALUES (%s, %s, %s, %s)"
        try:
            # Some attributes need to be casted to proper types
            session.execute(query, (line[1], line[4], line[9], int(line[10])))    
        except Exception as e:
            print(e)
        
# Get result from query to test table
try:
    rows = session.execute(query_fans)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.first_name, row.last_name, sep="--")
                    

Jacqueline--Lynch
Tegan--Levine
Sara--Johnson


### Drop the tables before closing out the sessions

In [14]:
# Drop tables
query = "DROP TABLE IF EXISTS songs_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "DROP TABLE IF EXISTS user_playlist"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "DROP TABLE IF EXISTS song_fans"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [15]:
session.shutdown()
cluster.shutdown()